In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
import yaml
from datasets import Dataset
from trl import KTOTrainer, KTOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

[2024-06-18 16:52:32,986] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [3]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'
checkpoint = yaml.safe_load(open('checkpoint_kto_dataset_python_cot.yaml'))
output_path = 'kto_output'
dataset = Dataset.from_dict(
    {
        'prompt': checkpoint['prompts'],
        'completion': checkpoint['completions'],
        'label': checkpoint['labels'],
    }
)

In [4]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype='bfloat16',
) 

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
model_ref = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
trainer = KTOTrainer(
    model,
    model_ref,
    args=KTOConfig(
        output_dir=output_path,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        bf16=True,
        gradient_accumulation_steps=8, 
        do_eval=True,
        logging_strategy='steps',
        save_strategy='steps',
        evaluation_strategy='steps',
        logging_steps=20,
        save_steps=20,
        eval_steps=20,
        num_train_epochs=10,
        load_best_model_at_end=True,
        warmup_steps=20,
        
        #loss_type='bco',
        desirable_weight=2.3,
        undesirable_weight=1.0,
    ),
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:464: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:474: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:504: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/4110 [00:00<?, ? examples/s]

Extracting KL train dataset:   0%|          | 0/4110 [00:00<?, ? examples/s]

Processing tokenized train dataset:   0%|          | 0/4110 [00:00<?, ? examples/s]

Processing tokenized train KL dataset:   0%|          | 0/4110 [00:00<?, ? examples/s]

...

Processing tokenized eval dataset:   0%|          | 0/457 [00:00<?, ? examples/s]

Processing tokenized eval KL dataset:   0%|          | 0/457 [00:00<?, ? examples/s]

Filtering desirable examples:   0%|          | 0/4110 [00:00<?, ? examples/s]

Filtering undesirable examples:   0%|          | 0/4110 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [6]:
trainer.train()

wandb: Currently logged in as: chengpong1127. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
